## Pandas Lab Assignment
#### Rachel Seo ydp7xv

#### **Data Set #1 Q1**: Are kickers more likely to miss the field goal when distance is 50 yds+?

#### Pseudocode 
1. Create a new dataset by filtering the current dataset for only the 'distance', 'GOOD', and 'Missed'
2. Extract all field goal attempts 
3. Separate attempts into two categories - shorter than 50 yards vs. 50 yards or more
4. Calculate miss percentage for each category 

In [62]:
# import libraries
import pandas as pd
import os
os.getcwd()

'/workspaces/DS-3021/02_function_basics'

#### Data Set #1

In [63]:
# loading and preparing the data
nfl = pd.read_csv("../data/nfl2008_fga.csv")

print(nfl.info())   # .info() outputs the structure of the dataframe object. It includes the columns, the data types in each, etc.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GameDate  1039 non-null   int64  
 1   AwayTeam  1039 non-null   object 
 2   HomeTeam  1039 non-null   object 
 3   qtr       1039 non-null   int64  
 4   min       1039 non-null   int64  
 5   sec       1039 non-null   int64  
 6   kickteam  1039 non-null   object 
 7   def       1039 non-null   object 
 8   down      1037 non-null   float64
 9   togo      1037 non-null   float64
 10  kicker    1039 non-null   int64  
 11  ydline    1039 non-null   int64  
 12  name      1039 non-null   object 
 13  distance  1039 non-null   int64  
 14  homekick  1039 non-null   int64  
 15  kickdiff  1039 non-null   int64  
 16  timerem   1039 non-null   int64  
 17  offscore  1039 non-null   int64  
 18  defscore  1039 non-null   int64  
 19  season    1039 non-null   int64  
 20  GOOD      1039 non-null   int6

In [64]:
# filtering dropping columns from dataset

column_drop = ['GameDate', 'AwayTeam','HomeTeam', 'qtr','min', 'sec', 'kickteam', 'def', 'down', 'togo', 'kicker', 'ydline', 'name','homekick', 'kickdiff', 'timerem','offscore','defscore','season', 'Blocked']
nfl_2 = nfl.drop(columns=column_drop)
nfl_2.dropna()

print(nfl_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   distance  1039 non-null   int64
 1   GOOD      1039 non-null   int64
 2   Missed    1039 non-null   int64
dtypes: int64(3)
memory usage: 24.5 KB
None


In [65]:
# creating new dataset for distances less than 50 yds
threshold_1 = 50
less_than_50 = nfl_2[nfl_2['distance'] < threshold_1]
less_than_50.head()

,distance,GOOD,Missed
0,30,1,0
1,46,1,0
2,28,1,0
3,37,1,0
4,39,1,0


In [66]:
missed_below_50 = less_than_50['Missed'].sum()
missed_percent_1 = (missed_below_50 / 1039) * 100  # 1039 is the total number of kicks recorded as seen in the nfl_2.info() printed above
print(missed_percent_1)

9.624639076034649


In [67]:
# creating new dataset for distances greater than and equal to 50 yds
threshold_2 = 50
greater_than_50 = nfl_2[nfl_2['distance'] >= threshold_2]
greater_than_50.head()

,distance,GOOD,Missed
13,51,1,0
14,51,1,0
17,50,0,1
30,51,1,0
31,51,1,0


In [68]:
missed_above_50 = greater_than_50['Missed'].sum()
missed_percent_2 = (missed_above_50 / 1039) * 100
print(missed_percent_2)

3.753609239653513


#### Complete Function

In [69]:
def missed_kicks(file_path, threshold=50):
    df = pd.read_csv(file_path)
    total_rows = df.shape[0]
    less_than_50 = df[df['distance'] < threshold]
    percent_missed_below_50 = (less_than_50['Missed'].sum() / total_rows)*100
    greater_than_50 = df[df['distance'] >= threshold]
    percent_missed_above_50 = (greater_than_50['Missed'].sum() / total_rows)*100
    return percent_missed_below_50, percent_missed_above_50

    

#### Answering Question #1:
According to this dataset, kickers missed more kicks when the distance was below 50 yards versus when the distance was at or above 50 yards.

In [70]:
print(missed_kicks(file_path = '../data/nfl2008_fga.csv', threshold=50))


(np.float64(9.624639076034649), np.float64(3.753609239653513))


#### **Data Set #1 Q2**: Which kicker(s) have a 100% completion rate?

#### Pseudocode 
1. Create a new dataset by filtering the current dataset for only the 'name', 'GOOD', and 'Missed'
2. Group by kicker
3. Calculate completion percentage for each kicker
4. Filter by completion rate to return only those with 100%

In [86]:
# filter columns 
column_drop = ['GameDate', 'AwayTeam','HomeTeam', 'qtr','min', 'sec', 'kickteam', 'def', 'down', 'togo', 'ydline','homekick', 'kickdiff', 'timerem','offscore','defscore','season', 'Blocked', 'distance', 'kicker']
nfl_k = nfl.drop(columns=column_drop)
nfl_k.dropna()

# create new column for all attempts
nfl_k['all_attempts'] = nfl_k['GOOD'] + nfl_k['Missed']

print(nfl_k.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          1039 non-null   object
 1   GOOD          1039 non-null   int64 
 2   Missed        1039 non-null   int64 
 3   all_attempts  1039 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 32.6+ KB
None


In [88]:
# group dataset by name and calculate the total attempts and completions 
kicker_stats = nfl_k.groupby('name').agg(
    total_attempts=('all_attempts', 'sum'),
    completions=('GOOD', 'sum')
).reset_index()

kicker_stats['completion_percentage'] = (kicker_stats['completions'] / kicker_stats['total_attempts']) * 100

print(kicker_stats)

            name  total_attempts  completions  completion_percentage
0    A.Vinatieri              26           22              84.615385
1        C.Barth              12           10              83.333333
2        D.Akers              50           45              90.000000
3    D.Carpenter              26           23              88.461538
4       D.Rayner               1            1             100.000000
5      G.Hartley              13           13             100.000000
6        J.Brown              36           31              86.111111
7       J.Carney              43           40              93.023256
8         J.Elam              32           30              93.750000
9        J.Feely              28           24              85.714286
10      J.Hanson              22           22             100.000000
11       J.Kasay              31           28              90.322581
12      J.Nedney              33           29              87.878788
13      J.Scobee              25  

In [94]:
# kicker(s) with 100% completion rate

kickers_with_100_percent = kicker_stats[kicker_stats['completion_percentage'] == 100]
print(kickers_with_100_percent)


         name  total_attempts  completions  completion_percentage
4    D.Rayner               1            1                  100.0
5   G.Hartley              13           13                  100.0
10   J.Hanson              22           22                  100.0
16    L.Tynes               1            1                  100.0


#### Complete Function

In [99]:
def kicker_completion(y):
    column_drop = ['GameDate', 'AwayTeam','HomeTeam', 'qtr','min', 'sec', 'kickteam', 'def', 'down', 'togo', 'ydline','homekick', 'kickdiff', 'timerem','offscore','defscore','season', 'Blocked', 'distance', 'kicker']
    y1 = y.drop(columns=column_drop)
    y1['all_attempts'] = y1['GOOD'] + nfl_k['Missed']
    y2 = y1.groupby('name').agg(total_attempts=('all_attempts', 'sum'), completions=('GOOD', 'sum')).reset_index()
    y2['completion_percentage'] = (y2['completions'] / y2['total_attempts']) * 100
    kicks_100 = y2[y2['completion_percentage'] == 100]
    return(kicks_100)
    

#### Answering Q2

In [100]:
print(kicker_completion(nfl))

         name  total_attempts  completions  completion_percentage
4    D.Rayner               1            1                  100.0
5   G.Hartley              13           13                  100.0
10   J.Hanson              22           22                  100.0
16    L.Tynes               1            1                  100.0


#### **Data Set #2 Q1**: What was the most popular genre of top songs released in years 2022 and 2023?

#### Pseudocode 
1. Filter the dataset for columns 'title', 'top genre', 'year', and 'popularity'
2. Filter and create a new dataset for years 2022 and 2023
3. Group by year and top genre
4. Calculate average popularity ratings for each genre in each year 
5. Calculate which genre has the highest popularity average in each year

#### Data Set #2

In [71]:
# loading and preparing the data
best_songs_spotify = pd.read_csv("../data/Best_Songs_Spotify_2000-2023.csv", sep=';',on_bad_lines='skip')

print(best_songs_spotify.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2385 entries, 0 to 2384
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2385 non-null   object
 1   artist         2385 non-null   object
 2   top genre      2385 non-null   object
 3   year           2385 non-null   int64 
 4   bpm            2385 non-null   int64 
 5   energy         2385 non-null   int64 
 6   danceability   2385 non-null   int64 
 7   dB             2385 non-null   int64 
 8   liveness       2385 non-null   int64 
 9   valence        2385 non-null   int64 
 10  duration       2385 non-null   int64 
 11  acousticness   2385 non-null   int64 
 12  speechiness    2385 non-null   int64 
 13  popularity     2385 non-null   int64 
dtypes: int64(11), object(3)
memory usage: 261.0+ KB
None


In [72]:
# filter dataset
filtered_best_songs = best_songs_spotify.loc[:, ['title', 'top genre', 'year', 'popularity']]

# new dataset for 2022 and 2023
filtered_best_songs2 = filtered_best_songs.query("year == 2022 or year == 2023")
filtered_best_songs2.head()

,title,top genre,year,popularity
0,Flowers,pop,2023,98
1,Cupid - Twin Ver.,k-pop girl group,2023,97
2,BESO,pop,2023,96
3,Boy's a liar Pt. 2,bronx drill,2023,96
4,Creepin' (with The Weeknd & 21 Savage),rap,2022,96


In [73]:
# grouping by year and top genre, then calculating the mean of 'popularity'
popular_genres = filtered_best_songs2.groupby(['year', 'top genre'])['popularity'].mean().reset_index()

# rename
popular_genres.columns = ['year', 'top_genre', 'average_popularity']

print(popular_genres)

    year                    top_genre  average_popularity
0   2022                    afrobeats           95.000000
1   2022                  alternative           29.000000
2   2022            argentine hip hop           93.000000
3   2022                      art pop           65.000000
4   2022                aussietronica           67.000000
5   2022                       azonto           34.000000
6   2022                  bedroom pop           94.000000
7   2022                     big room           94.000000
8   2022             canadian hip hop           90.000000
9   2022                colombian pop           90.000000
10  2022                    dance pop           72.400000
11  2022     deep underground hip hop           62.000000
12  2022                          edm           75.000000
13  2022                electro house           85.000000
14  2022                   electronic           35.000000
15  2022                  escape room           82.000000
16  2022      

In [74]:
# maximum average popularity for each year
max_popularity_per_year = popular_genres.groupby('year')['average_popularity'].max().reset_index()

# merging maximum avg popularity with the original grouped DataFrame to get the top genres
top_genres = pd.merge(max_popularity_per_year, popular_genres, on=['year', 'average_popularity'])

print(top_genres)

   year  average_popularity         top_genre
0  2022                96.0               rap
1  2023                97.0  k-pop girl group


#### Complete Function

In [75]:
def top_genres_by_year(x):
    x1 = x.loc[:, ['title', 'top genre', 'year', 'popularity']]
    x2 = x1.query("year == 2022 or year == 2023")
    x3 = x2.groupby(['year', 'top genre'])['popularity'].mean().reset_index()
    x3.columns = ['year', 'top_genre', 'average_popularity']
    max_pop = x3.groupby('year')['average_popularity'].max().reset_index()
    top_genres_by_year = pd.merge(max_pop, x3, on=['year', 'average_popularity'])
    return top_genres_by_year

#### Answering Q1

In [76]:
print(top_genres_by_year(best_songs_spotify))

   year  average_popularity         top_genre
0  2022                96.0               rap
1  2023                97.0  k-pop girl group


#### **Data Set #2 Q2**: Which artists had the highest number of top songs in the pop and rap genres?

#### Pseudocode 
1. Filter the dataset for columns 'title', 'artist', and 'top genre'
2. Filter and create new dataset for genres 'pop' and 'rap'
3. Group by top_genre and artist
4. Create a new column for the count of each artist (how many times they show up)
5. Calculate which artist has the most pop songs 
6. Calculate which artist has the most rap songs

In [105]:
# filter dataset
filtered_song = best_songs_spotify.loc[:, ['title', 'artist', 'top genre']]

# new dataset for pop and rap
pop_and_rap = filtered_song.query("`top genre` == 'pop' or `top genre` == 'rap'")
pop_and_rap.head()

,title,artist,top genre
0,Flowers,Miley Cyrus,pop
2,BESO,ROSALÍA,pop
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap
8,Anti-Hero,Taylor Swift,pop
11,Kill Bill,SZA,pop


In [108]:
# group by top genre and artist
genre_p = pop_and_rap.groupby(['top genre', 'artist']).size().reset_index(name='count')
genre_p.head()

,top genre,artist,count
0,pop,Alex & Sierra,1
1,pop,Anastacia,2
2,pop,André Aires,1
3,pop,Ariana Grande,20
4,pop,Ava Max,1


In [110]:
# artist with the highest pop songs 
pop_artist = genre_p[genre_p['top genre'] == 'pop'].sort_values(by='count', ascending=False).iloc[0]

# artist with the highest rap songs 
rap_artist = genre_p[genre_p['top genre'] == 'rap'].sort_values(by='count', ascending=False).iloc[0]

print(pop_artist)
print(rap_artist)


top genre             pop
artist       Taylor Swift
count                  23
Name: 58, dtype: object
top genre             rap
artist       Metro Boomin
count                   1
Name: 62, dtype: object


#### Complete Function

In [111]:
def top_pop_rap_artist(z):
    z1 = z.loc[:, ['title', 'artist', 'top genre']]
    z2 = z1.query("`top genre` == 'pop' or `top genre` == 'rap'")
    z3 = z2.groupby(['top genre', 'artist']).size().reset_index(name='count')
    p_artist = z3[z3['top genre'] == 'pop'].sort_values(by='count', ascending=False).iloc[0]
    r_artist = z3[z3['top genre'] == 'rap'].sort_values(by='count', ascending=False).iloc[0]
    return p_artist, r_artist

#### Answering Q2

In [113]:
print(top_pop_rap_artist(best_songs_spotify))

(top genre             pop
artist       Taylor Swift
count                  23
Name: 58, dtype: object, top genre             rap
artist       Metro Boomin
count                   1
Name: 62, dtype: object)
